In [ ]:
%cd /media/auto203/SSD2/JHY/python/enhanced-subject-verb-object-extraction-master


from typing import Tuple
from typing import List
from deepmultilingualpunctuation import PunctuationModel
from transformers import pipeline
import nltk
from Exiang import chinese_or_not
from Exiang import chinese_svo
import notam
import re
import warnings
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
from sentence_tools import read_words, get_item_pattern_list, pattern_combine, preprocess_sentence_code


S = """(A1912/15 NOTAMN
Q) LOVV/QWPLW/IV/BO/W/000/130/4809N01610E001
A) LOVV B) 1509261100 C) 1509261230
E) PJE WILL TAKE PLACE AT AREA LAAB IN WALDE
F) GND G) FL130)"""


warnings.filterwarnings("ignore")
MODEL = PunctuationModel()
RULES_TABLE = "NOTAM_table.xlsx"
action_words, reason_words, limit_words, source_words = read_words(path=RULES_TABLE, sheet_name='words_list')
pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls = get_item_pattern_list(action_words, reason_words, limit_words, source_words, path=RULES_TABLE, sheet_name='base_rules')
pattern_ls = pattern_combine(pattern_entity_ls, pattern_action_ls, pattern_reason_ls, pattern_limit_ls, pattern_source_ls)


# 解析 TODO: 去除非E项内容
def notam_parse_single(e_option_noisy: str) -> str:
    # 去除E):
    e_opition = e_option_noisy.replace('E)：', '')
    # 处理换行符
    e_opition = e_opition.rstrip('\n')
    e_opition = e_opition.replace('\n', '. ')
    # 去除括号里内容
    e_opition = re.sub('\(.*?\)', '', e_opition)
    # RWY和数字分开
    e_opition = re.sub(r"(Runway|RUNWAY|RWY|TWY)([0-9/]*)", r"\1 \2", e_opition)
    # 空格合并
    e_opition = ' '.join(e_opition.split())
    return e_opition

# 解码
def notam_decode(e_option: str) -> str:
    # 整理调包格式
    s_new = S.split("E)")[0] + "E) " + e_option + "\nF)" + S.split("E)")[1].split("F)")[1]
    res = notam.Notam.from_str(s_new)
    e_option_text = res.decoded().split("E)")[1].split("F)")[0].rstrip("\n").lstrip(' ').replace('\n', ' ')
    # 字典补充
    e_option_text = e_option_text.replace("MTOW", "Maximum takeoff weight")
    e_option_text = e_option_text.replace("ILS", "Instrument landing system")
    e_option_text = e_option_text.replace("IFR", "Instrument flight rules")
    # 原始符号处理
    e_option_text = e_option_text.replace(',', ", ")
    e_option_text = e_option_text.replace(':', ": ")
    e_option_text = ' '.join(e_option_text.split())
    return e_option_text

def bad_case_or_not(e_option_text: str) -> bool:
    return False

def bad_case_svo(e_option_text: str) -> List[List[str]]:
    return [[]]

# 标点
def punctuation(e_option_text: str) -> str:
    e_option_text_punc = MODEL.restore_punctuation(e_option_text)
    # 去掉RWY和数字之间的句号
    e_option_text_punc = re.sub(r"(Runway|RUNWAY|RWY|TWY).?", r"\1 ", e_option_text_punc)
    # 去掉数字和常见非开头词之间的句号
    e_option_text_punc = re.sub(r"([0-9LR/]+)[:\. ]*(?=Unserviceable|CLSD|Closed|CLS|closed)", r"\1 ", e_option_text_punc)
    # 合并空格
    e_option_text_punc = ' '.join(e_option_text_punc.split())
    # 去掉管理员批准中间的句号
    e_option_text_punc = re.sub(r"(Aerodrome)[\. ]*(OPERATOR (?:APPROVAL|Approved) ONLY)", r"\1 \2", e_option_text_punc)
    # 去掉通知时间中间的句号
    e_option_text_punc = re.sub(r"(MINUTES|Minutes|MIN|)[\. ]*((?:Prior|PRIOR) (?:NOTICE|Notice|NOTIFICATION|Permission|PERMISSION))", r"\1 \2", e_option_text_punc)
    # 去掉数字后冗余的句号
    e_option_text_punc = e_option_text_punc.replace("...", "呜呜呜")
    e_option_text_punc = e_option_text_punc.replace("..", '.')
    e_option_text_punc = e_option_text_punc.replace("呜呜呜", "...")
    # bad_cases
    e_option_text_punc = e_option_text_punc.replace("IS PROHIBITED. FROM INTERSECTION Taxiway C1.", "IS PROHIBITED FROM INTERSECTION Taxiway C1.")
    e_option_text_punc = e_option_text_punc.replace("CLOSED. ALL TRAINING AND VFR FLIGHTS.", "CLOSED ALL TRAINING AND VFR FLIGHTS.")
    e_option_text_punc = e_option_text_punc.replace("Refer To METHOD OF WORKING. PLAN 001-22, STAGE 2B.", "Refer To METHOD OF WORKING PLAN 001-22, STAGE 2B.")
    return e_option_text_punc

# 判断摘要
def summarization_or_not(e_option_text_punc: str) -> bool:
    return False

# 摘要
def summarization(text: str) -> str:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]["summary_text"]

# 分句
def sentence_tokenize(text: str) -> List[str]:
    sentences = []
    temp = ''
    # 加载punkt句子分割器
    sen_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    # 对句子进行分割
    sentences_ori = sen_tokenizer.tokenize(text)
    # 句子若仅含实体则与后一个句子合并
    for sentence_ori in sentences_ori:
        x = re.match(r"^(?:Runway|RUNWAY|RWY|TWY)[:\. ]?[0-9RL/]*\.$", sentence_ori)
        if x is None:
            sentences.append(temp + sentence_ori)
            temp = ''
        else:
            temp = sentence_ori
    return sentences

# 编码
def encode(sentence: str) -> str:
    sentence_code = sentence
    return sentence_code

# 单句解析
def sentence_parse(sentence_code: str):
    # preprocess sentence_code
    sentence_ls = preprocess_sentence_code(sentence_code, action_words)
    # sentence_ls = [sentence_code]
    is_match = False
    res_list_ls = []
    for sentence in sentence_ls:
        tmp_is_match = False
        res_dict = {item: "" for item in ['entity', 'action', 'reason', 'limit', 'limit_wings', 'limit_weight', 'source']}
        for pattern in pattern_ls:
            match = re.search(pattern, sentence, flags=re.I)
            if match:
                tmp_is_match = True
                match_dict = match.groupdict()
                # print(match_dict)
                if 'entity' in match_dict:
                    res_dict['entity'] = match_dict['entity']
                if 'entity_supply' in match_dict:
                    res_dict['entity'] = res_dict['entity'] + ' ' + match_dict['entity_supply']
                if 'action' in match_dict:
                    res_dict['action'] = match_dict['action']
                if 'reason' in match_dict:
                    res_dict['reason'] = match_dict['reason']
                if 'limit' in match_dict:
                    res_dict['limit'] = match_dict['limit']
                if 'source' in match_dict:
                    res_dict['source'] = match_dict['source']
                res_list = list(res_dict.values())
                res_list = [item.strip() for item in res_list]
                res_list_ls.append(res_list)
                break
        is_match = is_match or tmp_is_match
    return is_match, res_list_ls

# 合并居中单元格并保存
def to_merge(df, excel_name):
    # 按照'E项'列进行每行单元格合并
    # 'E项'列去重，确定一列需要合并成几个值
    df_key = list(set(df['E项'].values))
    wb = Workbook()
    ws = wb.active
    # 将每行数据写入ws中
    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)
    # 遍历去重后E项
    for i in df_key:
        # 获取E项等于指定值的几行数据
        df_id = df[df.E项 == i].index.tolist() # 索引值从0开始
        # 遍历，需要合并7列，openyxl中，读excel等的序号都是从1开始，所以合并7列，需要遍历range(1, 8)
        for j in range(1, 8):
            ws.merge_cells(start_row=df_id[0] + 2, end_row=df_id[-1] + 2, start_column=j, end_column=j) # 序号从1开始，所以行序号需要加2

    # save
    wb.save(excel_name)
    print('合并成功！')

In [2]:
# 读取excel数据
NOTAM = pd.read_excel("data.xlsx", sheet_name="相对简单")
for i, v in enumerate(NOTAM["E项"]):
    NOTAM["E项"][i] = notam_parse_single(v)

NOTAM["E项-人类语"] = "null"

for i, v in enumerate(NOTAM["E项"]):
    NOTAM["E项-人类语"][i] = notam_decode(v)


NOTAM

,类型,E项,E项-人类语,cache
0,跑道数据/限制,RWY 30 AVBL FOR TKOF AND LDG UNDER. FOLLOWING ...,Runway 30 Available FOR Take-off AND Landing U...,
1,跑道数据/限制,RWY 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,Runway 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,
2,跑道数据/限制,RWY 10R/28L STRENGTH DECREASED - SOAKED . RWY ...,Runway 10R/28L STRENGTH DECREASED - SOAKED . R...,
3,跑道数据/限制,"RWY 13/31 NARROWED TO 22,5M WHEN WET. FOR . LD...","Runway 13/31 NARROWED TO 22, 5M WHEN WET. FOR ...",
4,跑道数据/限制,RWY 13/31 USABLE FOR HELICOPTERS ONLY,Runway 13/31 USABLE FOR HELICOPTERS ONLY,
...,...,...,...,...
144,跑道数据/限制,RWY 17/35 EASTERN EDGE INTERFACE WITH RWY STRI...,Runway 17/35 EASTERN EDGE INTERFACE WITH Runwa...,
145,跑道数据/限制,"RWY 06/24 LTD, BUMPS PRESENCE. EXER CTN","Runway 06/24 Limited, BUMPS PRESENCE. Exercise...",
146,跑道数据/限制,RWY 17R/35L FULL STOP AND TOUCH AND GO NOT ALL...,Runway 17R/35L FULL STOP AND TOUCH AND GO NOT ...,
147,跑道数据/限制,UNPAVED RWY 17R/35L AND ULM STRIP 17L/35R CLSD...,UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R C...,


In [3]:
# 中间结果

NOTAM["E项-人类语标点符号预测"] = ''
NOTAM["E项-人类语分句"] = ''
NOTAM["是否使用分句及原因"] = ''

for i, v in enumerate(NOTAM["E项"]):
    if chinese_or_not(v):
        NOTAM["E项-人类语标点符号预测"][i] = v
        NOTAM["E项-人类语分句"][i] =v
    else:
        v = NOTAM["E项-人类语"][i]
        v_punct = punctuation(v)
        NOTAM["E项-人类语标点符号预测"][i] = v_punct
        sentences = sentence_tokenize(v_punct)
        NOTAM["E项-人类语分句"][i] = str(sentences)

NOTAM.to_excel("中间结果-简单.xlsx",index=False)


NOTAM

,类型,E项,E项-人类语,E项-人类语标点符号预测,E项-人类语分句
0,跑道数据/限制,RWY 30 AVBL FOR TKOF AND LDG UNDER. FOLLOWING ...,Runway 30 Available FOR Take-off AND Landing U...,Runway 30 Available FOR Take-off AND Landing U...,['Runway 30 Available FOR Take-off AND Landing...
1,跑道数据/限制,RWY 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,Runway 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,Runway 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS.,['Runway 11/29 CLOSED ALL TRAINING AND VFR FLI...
2,跑道数据/限制,RWY 10R/28L STRENGTH DECREASED - SOAKED . RWY ...,Runway 10R/28L STRENGTH DECREASED - SOAKED . R...,Runway 10R/28L STRENGTH DECREASED - SOAKED Run...,['Runway 10R/28L STRENGTH DECREASED - SOAKED R...
3,跑道数据/限制,"RWY 13/31 NARROWED TO 22,5M WHEN WET. FOR . LD...","Runway 13/31 NARROWED TO 22, 5M WHEN WET. FOR ...","Runway 13/31 NARROWED TO 22, 5M WHEN WET FOR L...","['Runway 13/31 NARROWED TO 22, 5M WHEN WET FOR..."
4,跑道数据/限制,RWY 13/31 USABLE FOR HELICOPTERS ONLY,Runway 13/31 USABLE FOR HELICOPTERS ONLY,Runway 13/31 USABLE FOR HELICOPTERS ONLY.,['Runway 13/31 USABLE FOR HELICOPTERS ONLY.']
...,...,...,...,...,...
144,跑道数据/限制,RWY 17/35 EASTERN EDGE INTERFACE WITH RWY STRI...,Runway 17/35 EASTERN EDGE INTERFACE WITH Runwa...,Runway 17/35 EASTERN EDGE INTERFACE WITH Runwa...,['Runway 17/35 EASTERN EDGE INTERFACE WITH Run...
145,跑道数据/限制,"RWY 06/24 LTD, BUMPS PRESENCE. EXER CTN","Runway 06/24 Limited, BUMPS PRESENCE. Exercise...",Runway 06/24: Limited BUMPS PRESENCE: Exercise...,['Runway 06/24: Limited BUMPS PRESENCE: Exerci...
146,跑道数据/限制,RWY 17R/35L FULL STOP AND TOUCH AND GO NOT ALL...,Runway 17R/35L FULL STOP AND TOUCH AND GO NOT ...,"Runway 17R/35L FULL STOP AND TOUCH AND GO, NOT...","['Runway 17R/35L FULL STOP AND TOUCH AND GO, N..."
147,跑道数据/限制,UNPAVED RWY 17R/35L AND ULM STRIP 17L/35R CLSD...,UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R C...,UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R C...,['UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R...


In [6]:
# 最终汇总

NOTAM["cache"] = ''

for i, v in enumerate(NOTAM["E项"]):
    if chinese_or_not(v):
        svo_all = chinese_svo(v)
    elif bad_case_or_not(v):
        svo_all = bad_case_svo(v)
    else:
        svo_all = []
        v = NOTAM["E项-人类语"][i]
        sentences = sentence_tokenize(punctuation(v))
        # 遍历一个E项所有待解析单句
        for k in sentences:
            # 把所有解析出来的结果汇总
            if sentence_parse(k)[0]:
                for m in sentence_parse(k)[1]:
                    svo_all.append(m)

    # svo_all 加到对应表格
    cache = ""
    for n in svo_all:
        # resul是一个解析结果（用in间隔）
        result_single = n[0] + "/in/" + n[1] + "/in/" + n[2] + "/in/"+ n[3] + "/in/" + n[4] + "/in/" + n[5] + "/in/" + n[6]
        cache += "/out/"
        cache += result_single
        cache = cache.lstrip("/out/")
    NOTAM["cache"][i] = cache

    # debug
    # print("第{}个E项解析出来的: ".format(i+1), svo_all)
    # if i == 11:
    #     break

# 拆分多个行
NOTAM["cache"] = NOTAM["cache"].str.split("/out/")
NOTAM = NOTAM.explode("cache")

# 拆分多个列
NOTAM_cache =NOTAM["cache"].str.split('/in/', expand=True)
NOTAM = NOTAM[~NOTAM.index.duplicated(keep="first")].drop(["cache"],axis=1).join(NOTAM_cache, how="right")

# rename
NOTAM.columns = ['类型', 'E项', 'E项-人类语', "E项-人类语标点符号预测", "E项-人类语分句", "是否使用分句及原因", '实体', '动作', "原因", "限制", "限制_翼展", "限制_重量", "来源"]

# 合并居中单元格并保存
excel_name = "相对简单.xlsx"
NOTAM.to_excel(excel_name, index=False)
to_merge(pd.read_excel(excel_name), excel_name)


NOTAM

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.457 seconds.
Prefix dict has been built successfully.


合并成功！


,类型,E项,E项-人类语,分句结果,限制,限制_翼展,限制_重量,实体,动作,动作后,原因,跑道实体
0,跑道数据/限制,RWY 30 AVBL FOR TKOF AND LDG UNDER. FOLLOWING...,Runway 30 Available FOR Take-off AND Landing U...,['Runway 30 Available FOR Take-off AND Landing...,,,,rwy 30,available,for take-off and ldg under following conditio...,,rwy 30
1,跑道数据/限制,RWY 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,Runway 11/29 CLOSED ALL TRAINING AND VFR FLIGHTS,"['Runway 11/29 CLOSED.', 'ALL TRAINING AND VFR...",,,,rwy 11/29,closed,,,rwy 11/29
2,跑道数据/限制,RWY 10R/28L STRENGTH DECREASED - SOAKED . RWY....,Runway 10R/28L STRENGTH DECREASED - SOAKED . R...,['Runway 10R/28L STRENGTH DECREASED - SOAKED R...,,,,,None,None,None,None
3,跑道数据/限制,"RWY 13/31 NARROWED TO 22,5M WHEN WET. FOR . LD...","Runway 13/31 NARROWED TO 22,5M WHEN WET. FOR ....","['Runway 13/31 NARROWED TO 22,5M WHEN WET FOR ...",,,,rwy 13/31,narrowed to,"22,5m when wet for landing/take-off use south...",,"rwy 13/31', 'rwy"
4,跑道数据/限制,RWY 13/31 USABLE FOR HELICOPTERS ONLY,Runway 13/31 USABLE FOR HELICOPTERS ONLY,['Runway 13/31 USABLE FOR HELICOPTERS ONLY.'],,,,rwy 13/31,usable,for helicopters only,,rwy 13/31
...,...,...,...,...,...,...,...,...,...,...,...,...
144,跑道数据/限制,RWY 17/35 EASTERN EDGE INTERFACE WITH RWY STRI...,Runway 17/35 EASTERN EDGE INTERFACE WITH Runwa...,['Runway 17/35 EASTERN EDGE INTERFACE WITH Run...,,,,,None,None,None,None
145,跑道数据/限制,"RWY 06/24 LTD, BUMPS PRESENCE. EXER CTN","Runway 06/24 Limited, BUMPS PRESENCE. Exercise...",['Runway 06/24: Limited BUMPS PRESENCE: Exerci...,,,,,None,None,None,None
146,跑道数据/限制,RWY 17R/35L FULL STOP AND TOUCH AND GO NOT ALL...,Runway 17R/35L FULL STOP AND TOUCH AND GO NOT ...,"['Runway 17R/35L FULL STOP AND TOUCH AND GO, N...",,,,,None,None,None,None
147,跑道数据/限制,UNPAVED RWY 17R/35L AND ULM STRIP 17L/35R CLSD...,UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R C...,['UNPAVED Runway 17R/35L AND ULM STRIP 17L/35R...,,,,npaved rwy 17r/35l and ulm strip 17l/35r,closed,,due work inpr,rwy 17r/35l
